In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import datetime

In [2]:
%env SQLA_GOST_DATABASE_NAME=gost
%env SQLA_GOST_DATABASE_HOST=localhost
%env SQLA_GOST_DATABASE_PORT=5432
%env SQLA_GOST_DATABASE_USER=postgres
%env SQLA_GOST_DATABASE_PASS=postgres

env: SQLA_GOST_DATABASE_NAME=gost
env: SQLA_GOST_DATABASE_HOST=localhost
env: SQLA_GOST_DATABASE_PORT=5432
env: SQLA_GOST_DATABASE_USER=postgres
env: SQLA_GOST_DATABASE_PASS=postgres


In [3]:
# SQLAlchemy + psycopg2 connection
sqla_endpoint ="postgresql+psycopg2://{}:{}@{}:{}/{}".format(
    os.environ['SQLA_GOST_DATABASE_USER'],
    os.environ['SQLA_GOST_DATABASE_PASS'],
    os.environ['SQLA_GOST_DATABASE_HOST'],
    os.environ['SQLA_GOST_DATABASE_PORT'],
    os.environ['SQLA_GOST_DATABASE_NAME'],
)
engine = create_engine(sqla_endpoint)

In [4]:
class Config():
    def __init__(self):
        table_names = {
            'strain1' : 'sensor_00158d00000e0ee5',
            'acc1' : 'sensor_00158d00000e0fe9',
            'acc2' : 'sensor_00158d00000e054c',
            'incl' : 'sensor_00158d00000e1024',
            'temp' : 'sensor_00158d00000e047b',
            'strain2': 'sensor_000000008bff436'
        }
        self.table_names = table_names
        
        self.column_names = {
            table_names['strain1'] : ['id','ts','ch_mv0','ch_mv1','ch_mv2','ch_mv3'],
            table_names['acc1'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['acc2'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['incl'] : ['id','ts','ch_x','ch_y'],
            table_names['temp'] : ['ch_temperature'],
            table_names['strain2'] : ['id','ts','ch_mv0','ch_mv0_379']
        }

        self.schema = 'v1'   
        self.dateformat = '%Y-%m-%d' #%H:%M:%S'
config = Config()

In [6]:
class Settings():
    def __init__(self):
        self.sensors = ['acc1']
        self.n_samples = 10
        self.start_date = '2020-10-27'
        self.end_date = '2020-11-07'
settings = Settings()

In [7]:
class QueryGenerator():
    def __init__(self,sensors, start_date, end_date):
        self.sensors = sensors
        self.start_date = start_date
        self.end_date = end_date
        
        
    def generate_select(self):
        select_command = ''
        for sensor in self.sensors:
            table = config.table_names[sensor]
            for column in config.column_names[table]:
                select_command+=table+'.'+column+' AS '+sensor+'_'+column+' ,' 
        select_command += f"{config.table_names[self.sensors[0]]}.ts AS ts "
        return select_command

    def generate_where_id(self):
        where_clause = f"{config.schema}.{config.table_names[self.sensors[0]]}.id < {self.settings.n_samples}"
        return where_clause

    def generate_where(self,table_name=config.table_names['acc1']):
        where_clause = f"{config.schema}.{table_name}.ts BETWEEN \'{self.parse_date(self.start_date)}\' AND \'{self.parse_date(self.end_date)}\' "
        return where_clause

    def generate_and(self):
        and_clause = ''
        if len(self.sensors)>1:
            for i in range(len(self.sensors)-1):
                and_clause += f"AND {config.schema}.{config.table_names[self.sensors[0]]}.ts = {config.schema}.{config.table_names[self.sensors[i+1]]}.ts "
                # ts ensures integrity in data
        return and_clause

    def parse_date(self,date):
        
        return str(datetime.datetime.strptime(date,config.dateformat))

    def generate_query(self):
        query = ''
        query += f"SELECT {self.generate_select()}"
        query += f"FROM {config.schema}.{(', '+config.schema+'.').join([config.table_names[sensor] for sensor in self.sensors])} "
        query += f"WHERE {self.generate_where()}"
        query += self.generate_and()
        return query
        
    def generate_temp_query(self):
        query = ''
        query += f"SELECT {config.table_names['temp']}.{''.join(config.column_names[config.table_names['temp']])} AS temp "
        query += f"FROM {config.schema}.{config.table_names['temp']} "
        query += f"WHERE {self.generate_where(table_name = config.table_names['temp'])}"
        return query
    
    def generate_latest_query(self,steps=50): # Needs a model object to figure out how many tuples to request
        query = ''
        query += f" SELECT {self.generate_select()}"
        query += f" FROM {config.schema}.{(', '+config.schema+'.').join([config.table_names[sensor] for sensor in self.sensors])} "
        query += f" ORDER BY id DESC LIMIT {steps} "
        return query

In [8]:
config = Config()
settings = Settings()
query_generator = QueryGenerator(settings.sensors,settings.start_date,settings.end_date)

In [9]:
query_generator.generate_query()

"SELECT sensor_00158d00000e0fe9.id AS acc1_id ,sensor_00158d00000e0fe9.ts AS acc1_ts ,sensor_00158d00000e0fe9.ch_x AS acc1_ch_x ,sensor_00158d00000e0fe9.ch_y AS acc1_ch_y ,sensor_00158d00000e0fe9.ch_z AS acc1_ch_z ,sensor_00158d00000e0fe9.ts AS ts FROM v1.sensor_00158d00000e0fe9 WHERE v1.sensor_00158d00000e0fe9.ts BETWEEN '2020-10-27 00:00:00' AND '2020-11-07 00:00:00' "

In [10]:
df = pd.read_sql_query(
    sql = query_generator.generate_query(),
    con = sqla_endpoint
)

In [11]:
df

,acc1_id,acc1_ts,acc1_ch_x,acc1_ch_y,acc1_ch_z,ts
0,1,2020-10-27 07:59:54.000000,0.07814,0.37655,0.92338,2020-10-27 07:59:54.000000
1,2,2020-10-27 07:59:54.030303,0.07776,0.37586,0.92315,2020-10-27 07:59:54.030303
2,3,2020-10-27 07:59:54.060606,0.07829,0.37563,0.92110,2020-10-27 07:59:54.060606
3,4,2020-10-27 07:59:54.090909,0.07837,0.37594,0.92315,2020-10-27 07:59:54.090909
4,5,2020-10-27 07:59:54.121212,0.07723,0.37548,0.92193,2020-10-27 07:59:54.121212
...,...,...,...,...,...,...
1526245,1526246,2020-11-06 21:01:43.848511,0.07943,0.37624,0.92140,2020-11-06 21:01:43.848511
1526246,1526247,2020-11-06 21:01:43.878784,0.07951,0.37617,0.92171,2020-11-06 21:01:43.878784
1526247,1526248,2020-11-06 21:01:43.909119,0.07943,0.37586,0.92155,2020-11-06 21:01:43.909119
1526248,1526249,2020-11-06 21:01:43.939392,0.07913,0.37571,0.92110,2020-11-06 21:01:43.939392
